In [1]:
from notebook_functions import *
from config import *
import pandas as pd
import eyed3, json

# suppress eyed3 'non-standard genre' warnings
eyed3.log.setLevel("ERROR")

# dataset to re-label location
root_dir = z_path + "\\fma_medium"


### load data for re-labeling

In [2]:
# navigate to metadata for re-labeling
metadata = z_path + "\\fma_metadata"
os.chdir(metadata)

# import raw track data & make dataframes
genres_df = pd.read_csv('genres.csv')
tracks_df = pd.read_csv('raw_tracks.csv')

# list of full file paths
song_paths = make_file_list(fma_dir)

C:\Users\Zack\Desktop\test_space\093\093015.mp3
093015


In [3]:
# limit to just the relevant details
tracks_df = tracks_df[["track_id", "track_bit_rate", "track_genres"]]

# drop rows missing the 'genre' value
tracks_df = tracks_df[tracks_df['track_genres'].notna()]


,track_id,track_bit_rate,track_genres
0,2,256000.0,"[{'genre_id': '21', 'genre_title': 'Hip-Hop', ..."
1,3,256000.0,"[{'genre_id': '21', 'genre_title': 'Hip-Hop', ..."
2,5,256000.0,"[{'genre_id': '21', 'genre_title': 'Hip-Hop', ..."
3,10,192000.0,"[{'genre_id': '10', 'genre_title': 'Pop', 'gen..."
4,20,256000.0,"[{'genre_id': '76', 'genre_title': 'Experiment..."


In [4]:
# string conversion to list of genre dictionaries
tracks_df['track_genres'] = tracks_df['track_genres'].apply(eval)

In [5]:
# assign primary genre classification id, then make an int
tracks_df['genre_ids'] = tracks_df['track_genres'].apply(get_genre_id)
tracks_df['genre_ids'] = tracks_df['genre_ids'].apply(id_to_int)

In [6]:
tracks_df['track_id'] = tracks_df['track_id'].apply(id_to_int)

In [7]:
genres_df.head()

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5


### create lookups

In [8]:
# dictionaries for top-level genre lookup from ID values
genre_lookup = genres_df.set_index('genre_id').to_dict()['title']

# dictionary for track ID to genre ID
track_genre_lookup = tracks_df.set_index('track_id').to_dict()['genre_ids']

### edit mp3 meta-data to have correct genre assignments

In [9]:

for i in range(len(song_paths)):
    
    # get mp3 filename for ID reference and check to skip
    if song_paths[i][-10:] not in skip_list:

        # load audio file
        audiofile = eyed3.load(song_paths[i])

        # cross-reference with CSV id values and assign to audio file
        audiofile.tag.genre = genre_lookup[track_genre_lookup[int(song_paths[i][-10:-4])]]
        audiofile.tag.save()
print("all done!")

all done!
